In [1]:
# Import the needed libraries
import pandas as pd
from Bio import SeqIO

In [ ]:
# Define the dataframes that will have the calculated feature values
unaligned_data = {"ID": [], 
                  "Sequence": [], 
                  "Amino Acid Composition": [], 
                  "Physicochemical Properties": [], 
                  "Sequence Length": [], 
                  "Sequence Motif": []}

aligned_data = {"ID": [], 
                "Aligned Sequence": [],
                "Consensus Sequence": [], 
                "Profile-Based Features": [],
                "Secondary Structure Predictions": [],
                "Gap Statistics": []}

for seq_record in SeqIO.parse("sequences.fasta", "fasta"):
    unaligned_data["ID"].append(seq_record.id)
    unaligned_data["Unaligned Sequence"].append(seq_record.seq)
    
for seq_record in SeqIO.parse("aligned_sequences.fasta", "fasta"):
    aligned_data["ID"].append(seq_record.id)
    aligned_data["Aligned Sequence"].append(seq_record.seq)

unaligned_df = pd.DataFrame(unaligned_data)
unaligned_df.head()

aligned_df = pd.DataFrame(aligned_data)
aligned_df.head()